In [0]:
# Importing necessary files.
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
# Downloading the inception v3 weights.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Importing the inception model.  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = "/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Loading the pretrained weights in the model.
pre_trained_model.load_weights(local_weights_file)

# Making all the layers in the pre-trained model non-trainable.
for layer in pre_trained_model.layers:
  layer.trainable = False
  
# Printing the model summary.
pre_trained_model.summary()

# Expected Output is extremely large, but should end with:

#batch_normalization_v1_281 (Bat (None, 3, 3, 192)    576         conv2d_281[0][0]                 
#__________________________________________________________________________________________________
#activation_273 (Activation)     (None, 3, 3, 320)    0           batch_normalization_v1_273[0][0] 
#__________________________________________________________________________________________________
#mixed9_1 (Concatenate)          (None, 3, 3, 768)    0           activation_275[0][0]             
#                                                                 activation_276[0][0]             
#__________________________________________________________________________________________________
#concatenate_5 (Concatenate)     (None, 3, 3, 768)    0           activation_279[0][0]             
#                                                                 activation_280[0][0]             
#__________________________________________________________________________________________________
#activation_281 (Activation)     (None, 3, 3, 192)    0           batch_normalization_v1_281[0][0] 
#__________________________________________________________________________________________________
#mixed10 (Concatenate)           (None, 3, 3, 2048)   0           activation_273[0][0]             
#                                                                 mixed9_1[0][0]                   
#                                                                 concatenate_5[0][0]              
#                                                                 activation_281[0][0]             
#==================================================================================================
#Total params: 21,802,784
#Trainable params: 0
#Non-trainable params: 21,802,784

--2019-05-02 18:14:46--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   175MB/s    in 0.5s    

2019-05-02 18:14:47 (175 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (

In [3]:
# Choosing a layer to be our last layer for our network.
last_layer = pre_trained_model.get_layer("mixed7")
print("Last layer output shape: ", last_layer.output_shape)
last_output = last_layer.output

# Expected Output:
# ('last layer output shape: ', (None, 7, 7, 768))

('Last layer output shape: ', (None, 7, 7, 768))


In [0]:
# Defining a Callback class that stops training once accuracy reaches 99.9%.
class myCallback(tf.keras.callbacks.Callback):
  
  def on_epoch_end(self, epoch, logs={}):
    """
    Called at the end of each epoch.
    """
    if(logs.get("acc")>0.999):
      print("\nReached 99.9% accuracy, so cancelling training!")
      self.model.stop_training = True

In [5]:
# Flattening the output layer to 1 dimension.
x = layers.Flatten()(last_output)

# Adding a fully connected layer with 1,024 hidden units and ReLU activation.
x = layers.Dense(1024, activation="relu")(x)

# Adding a dropout rate of 20%.
x = layers.Dropout(0.2)(x)   
  
# Adding a final sigmoid layer for classification.
x = layers.Dense(1, activation="sigmoid")(x)    

# Instatiating model. '.input' contains input information for our model.
model = Model(pre_trained_model.input, x) 

model.compile(optimizer = "adam", 
              loss = "binary_crossentropy", 
              metrics = ["acc"])

model.summary()

# Expected output will be large. Last few lines should be:

# mixed7 (Concatenate)            (None, 7, 7, 768)    0           activation_248[0][0]             
#                                                                  activation_251[0][0]             
#                                                                  activation_256[0][0]             
#                                                                  activation_257[0][0]             
# __________________________________________________________________________________________________
# flatten_4 (Flatten)             (None, 37632)        0           mixed7[0][0]                     
# __________________________________________________________________________________________________
# dense_8 (Dense)                 (None, 1024)         38536192    flatten_4[0][0]                  
# __________________________________________________________________________________________________
# dropout_4 (Dropout)             (None, 1024)         0           dense_8[0][0]                    
# __________________________________________________________________________________________________
# dense_9 (Dense)                 (None, 1)            1025        dropout_4[0][0]                  
# ==================================================================================================
# Total params: 47,512,481
# Trainable params: 38,537,217
# Non-trainable params: 8,975,264


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization_v1[0][0]

In [6]:
# Geting the Horse or Human dataset.
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip -O /tmp/horse-or-human.zip

# Geting the Horse or Human Validation dataset.
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip -O /tmp/validation-horse-or-human.zip 
  
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

# Unzipping training dataset.
zip_file = zipfile.ZipFile("/tmp/horse-or-human.zip", "r")
zip_file.extractall("/tmp/training")
zip_file.close()

# Unzipping validation dataset.
zip_file = zipfile.ZipFile("/tmp/validation-horse-or-human.zip", "r")
zip_file.extractall("/tmp/validation")
zip_file.close()

--2019-05-02 18:15:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 2607:f8b0:4001:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   134MB/s    in 1.1s    

2019-05-02 18:15:03 (134 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2019-05-02 18:15:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/z

In [7]:
# Defining our example directories and files
train_dir = '/tmp/training'
validation_dir = '/tmp/validation'

train_horses_dir = os.path.join(train_dir, "horses")
train_humans_dir = os.path.join(train_dir, "humans")
validation_horses_dir = os.path.join(validation_dir, "horses")
validation_humans_dir = os.path.join(validation_dir, "humans")

train_horses_fnames = os.listdir(train_horses_dir)
train_humans_fnames = os.listdir(train_humans_dir)
validation_horses_fnames = os.listdir(validation_horses_dir)
validation_humans_fnames = os.listdir(validation_humans_dir)

print("Number of training horse images: {}".format(len(train_horses_fnames)))
print("Number of training human images: {}".format(len(train_humans_fnames)))
print("Number of validation horse images: {}".format(len(validation_horses_fnames)))
print("Number of validation human images: {}".format(len(validation_humans_fnames)))

# Expected Output:
# 500
# 527
# 128
# 128

500
527
128
128


In [10]:
# Adding our data-augmentation parameters to ImageDataGenerator
training_data_generator = ImageDataGenerator(rescale = 1/255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_data_generator = ImageDataGenerator(rescale = 1/255)

# Flow training images in batches of 20 using train_datagen generator
training_data_generator = training_data_generator.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = "binary", 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_data_generator =  validation_data_generator.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = "binary", 
                                                          target_size = (150, 150))

# Expected Output:
# Found 1027 images belonging to 2 classes.
# Found 256 images belonging to 2 classes.

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [11]:
# Run this and see how many epochs it should take before the callback
# fires, and stops training at 99.9% accuracy
# (It should take less than 100 epochs)
callbacks = myCallback()
history = model.fit_generator(
            training_data_generator,
            validation_data = validation_data_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 2,
            callbacks=[callbacks])

Epoch 1/100
13/13 [==============================] - 1s 112ms/step - loss: 7.1650e-05 - acc: 1.0000
 - 12s - loss: 0.1601 - acc: 0.9552 - val_loss: 7.1650e-05 - val_acc: 1.0000
Epoch 2/100
13/13 [==============================] - 1s 109ms/step - loss: 5.6619e-05 - acc: 1.0000
 - 12s - loss: 0.0451 - acc: 0.9825 - val_loss: 5.6619e-05 - val_acc: 1.0000
Epoch 3/100
13/13 [==============================] - 1s 107ms/step - loss: 0.0024 - acc: 1.0000
 - 12s - loss: 0.0661 - acc: 0.9834 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 4/100
13/13 [==============================] - 1s 105ms/step - loss: 6.8179e-04 - acc: 1.0000
 - 12s - loss: 0.1007 - acc: 0.9727 - val_loss: 6.8179e-04 - val_acc: 1.0000
Epoch 5/100
13/13 [==============================] - 1s 114ms/step - loss: 0.0018 - acc: 1.0000
 - 13s - loss: 0.0338 - acc: 0.9844 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 6/100
13/13 [==============================] - 1s 110ms/step - loss: 0.1035 - acc: 0.9805
 - 12s - loss: 0.0741 - acc: 0.9825

In [0]:
# Plotting findings.
import matplotlib.pyplot as plt
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs, acc, "r", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend(loc=0)
plt.show()